In [9]:
import requests
from requests.adapters import Retry, HTTPAdapter
import pandas as pd

In [10]:
av_key = 'FYBSO7S5U1F1KKO2'

In [11]:
av_request = requests.get(f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=TSCO.LON&outputsize=full&apikey={av_key}')
data = av_request.json()

In [12]:
data

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'TSCO.LON',
  '3. Last Refreshed': '2024-07-18',
  '4. Output Size': 'Full size',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2024-07-18': {'1. open': '323.5000',
   '2. high': '324.9000',
   '3. low': '321.2000',
   '4. close': '324.9000',
   '5. volume': '18324530'},
  '2024-07-17': {'1. open': '318.4000',
   '2. high': '321.6000',
   '3. low': '316.3000',
   '4. close': '321.6000',
   '5. volume': '31080090'},
  '2024-07-16': {'1. open': '314.7000',
   '2. high': '319.8000',
   '3. low': '312.8000',
   '4. close': '318.9000',
   '5. volume': '13011920'},
  '2024-07-15': {'1. open': '317.5000',
   '2. high': '320.3000',
   '3. low': '314.6000',
   '4. close': '315.7000',
   '5. volume': '11506730'},
  '2024-07-12': {'1. open': '315.9000',
   '2. high': '318.3000',
   '3. low': '312.1000',
   '4. close': '318.3000',
   '5. volume': '14951070'},
  '2024-07-11': {'1. open'

In [13]:
df = pd.DataFrame.from_dict(data['Time Series (Daily)'], orient='index').reset_index()
df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,Open,High,Low,Close,Volume
0,2024-07-18,323.5000,324.9000,321.2000,324.9000,18324530
1,2024-07-17,318.4000,321.6000,316.3000,321.6000,31080090
2,2024-07-16,314.7000,319.8000,312.8000,318.9000,13011920
3,2024-07-15,317.5000,320.3000,314.6000,315.7000,11506730
4,2024-07-12,315.9000,318.3000,312.1000,318.3000,14951070
...,...,...,...,...,...,...
4932,2005-01-10,319.7501,320.5001,319.0001,320.2501,11117302
4933,2005-01-07,317.7501,321.2501,317.7501,319.7501,37111214
4934,2005-01-06,315.0001,320.0001,314.7501,318.7501,17598422
4935,2005-01-05,315.5001,318.0001,315.0001,315.0001,28205028


In [14]:
df.head()
parquet_file = df.to_parquet('data.parquet')
json_file = df.to_json('json_file.json')

In [15]:
import csv
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = f'https://www.alphavantage.co/query?function=LISTING_STATUS&state=active&apikey={av_key}'

key_df = pd.DataFrame(columns=['Symbol'])

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list[1:]:
        symbol = row[0]
        temp = pd.DataFrame({'Symbol': symbol}, columns=['Symbol'], index=[0])
        key_df = pd.concat([key_df, temp], ignore_index=True)
        
key_df

,Symbol
0,A
1,AA
2,AAA
3,AAAU
4,AACG
...,...
11610,ZYME
11611,ZYNE
11612,ZYXI
11613,ZZK


In [16]:
df = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])

def get_stock_data(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={av_key}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for key in data['Time Series (Daily)']:
            temp = data['Time Series (Daily)'][key]
            date = key
            open = temp['1. open']
            high = temp['2. high']
            low = temp['3. low']
            close = temp['4. close']
            volume = temp['5. volume']
            temp_df = pd.DataFrame({'Date': date, 'Open': open, 'High': high, 'Low': low, 'Close': close, 'Volume': volume}, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], index=[0])
            pd.concat([df, temp_df], ignore_index=True)
            
for symbol in key_df['Symbol'][:500]:
    get_stock_data(symbol)
    
len(df)

KeyError: 'Time Series (Daily)'

In [ ]:
import requests
import json
import time

# Define the URL for the API endpoint
url = "https://query1.finance.yahoo.com/v7/finance/quote"

# Define the parameters for the request
params = {
    'symbols': 'AAPL'  # Example symbol for Apple Inc.
}

# Function to fetch data from Yahoo Finance
def fetch_data(url, params, retries=3, delay=5):
    for attempt in range(retries):
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            print("Rate limit exceeded. Retrying in {} seconds...".format(delay))
            time.sleep(delay)
        else:
            print(f"Failed to retrieve data: {response.status_code}")
            return None
    return None

# Fetch data with retries
data = fetch_data(url, params)

# Check if data was successfully retrieved
if data:
    print(json.dumps(data, indent=4))
else:
    print("Failed to retrieve data after retries.")


Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Failed to retrieve data after retries.


In [ ]:
from retry_logic import make_requests


In [ ]:
import requests

headers = {
        'Content-Type': 'application/json'
        }
requestResponse = requests.get("https://api.tiingo.com/tiingo/daily/aapl/prices?startDate=1990-01-01&token=006fb0b44d18cf7ab2fe767af8c7d315dd74c012", headers=headers)

data = requestResponse.json()
len(data)

8699

In [ ]:
import resend

resend.api_key = "re_EMsEH8fy_EkKMFhwyEwSA6QqTmAu3Q46K"

r = resend.Emails.send({
  "from": "onboarding@resend.dev",
  "to": "scarletmoon2003@gmail.com",
  "subject": "Hello World",
  "html": "<p>Congrats on sending your <strong>first email</strong>!</p>"
})

In [ ]:
from minio import Minio
import urllib3

In [1]:
from dotenv import dotenv_values
config = dotenv_values("minio.env")
config

OrderedDict([('username', 'admin'),
             ('password', 'abcdxyz1234567890'),
             ('access_key', 'iZDIWzUpyyiyqr0VAJxA'),
             ('secret_key', 'nESnAazgLSBdexz5hvkEi9LxSb0Ug1fAcrAI42e0'),
             ('server_url', 'localhost:9000')])

In [3]:
with open('minio.env', 'r') as f:
    lines = f.readlines()
    access_key = lines[0].split('=')[1].replace('\n', '')
    secure_key = lines[1].split('=')[1].replace('\n', '')

access_key
secure_key

'abcdxyz1234567890'

In [ ]:
client_2 = Minio("localhost:39000",
        access_key=access_key,
        secret_key=secure_key,
        secure=False
    )

In [ ]:
print(client_2.bucket_exists("tranduc1901"))

# client.make_bucket("python-test-bucket-3", location="us-east-1", object_lock=False)

True


In [ ]:
objects = client_2.list_objects(bucket_name="python-test-bucket-2", prefix=None)
for obj in objects:
    print(obj)

In [ ]:
import looker_sdk
import os
import json

In [ ]:
import requests

response = requests.get('http://localhost:39001')
response.status_code

200

In [30]:
from minio import Minio

class MinioClient:
    
    def __init__(self, host, port, access_key, secret_key):
        self.host = host
        self.port = port
        self.access_key = access_key
        self.secret_key = secret_key
        self.client = Minio(
            endpoint=f"{self.host}:{self.port}",
            access_key=self.access_key,
            secret_key=self.secret_key,
            secure=False,
        )
     

    # MinIO client function
    def _client(self):
        return self.client
        
    # Check if bucket exists
    def check_bucket(self, bucket_name):
        return self.client.bucket_exists(bucket_name=bucket_name)
    
    # PUT object function
    def put_object(self, bucket_name, obj_name, obj_file):
        if not self.check_bucket(bucket_name):
            # self.client.make_bucket(bucket_name=bucket_name, location="us-east-1", object_lock=False)
            raise ValueError("Bucket does not exist")
        self.client.fput_object(bucket_name=bucket_name, object_name=obj_name, file_path=obj_file)
        
    # GET object function
    def get_object(self, bucket_name, obj_name, file_path):
        if not self.check_bucket(bucket_name):
            return None
    
        response = self.client.fget_object(bucket_name=bucket_name, object_name=obj_name, file_path=file_path)            
        return response
        
    # DELETE object function
    def delete_object(self, bucket_name, obj_name):
        if not self.check_bucket(bucket_name):
            return None
        self.client.remove_object(bucket_name=bucket_name, object_name=obj_name)
        return 1
    
    # DELETE object list function
    def delete_objects(self, bucket_name, obj_name_list):
        if not self.check_bucket(bucket_name):
            return None
        self.client.remove_objects(bucket_name=bucket_name, delete_object_list=obj_name_list)
        return 1
    
    # # UPDATE object function
    # def update_object(self, bucket_name, obj_name):
    #     minio_client = self._client()
    #     if not self.check_bucket(bucket_name):
    
    # LIST object in a bucket
    def list_objects(self, bucket_name):
        if not self.check_bucket(bucket_name):
            print("Bucket does not exist")
            return None
        return [obj.object_name for obj in self.client.list_objects(bucket_name=bucket_name, prefix="")]
        
    

In [31]:


client_1 = MinioClient("localhost", "39000", access_key, secure_key)
objects = client_1.list_objects(bucket_name="python-test-bucket-2")
objects

['(The Practical Guides) Michael J. Donahoo, Gregory D. Speegle - SQL_ Practical Guide for Developers-Morgan Kaufmann (2005).pdf',
 '(Undergraduate Topics in Computer Science) Wolfgang Ertel (auth.) - Introduction to Artificial Intelligence-Springer-Verlag London (2011).pdf',
 '04725791.pdf',
 '1. Michael T. Goodrich, Roberto Tamassia, Michael H. Goldwasser - Data Structures and Algorithms in Java-Wiley (2014).pdf',
 '1512.02325v5.pdf',
 '2002.08155v4.pdf',
 '2009.01325.pdf',
 '2020.findings-emnlp.92.pdf',
 '21002117_L-ng-cAnh_BT1.pdf',
 '21002117_buoi4.2.zip',
 '21002134_TranMinhDuc.pdf',
 '2210.08176.pdf',
 '2305.15813v1.pdf',
 '2309.02033.pdf',
 '2_E30 Parallel maze generation.pdf',
 '3.MapReduce_introduction.pdf',
 '337430.pdf',
 '349 - Lich phat CC GDQP-AN.pdf',
 '4.1 Tệp và quản lý lỗi.pdf',
 '4.2 Lớp - OOP.pdf',
 '4glm_3_beamer_post.pdf',
 '591_oring.pdf',
 '6ab3ab2cec91ed4a5a2bde7e857aefdad149.pdf',
 '9367-13-12895-1-2-20201228.pdf',
 'ACO_Intro.pdf',
 'A_Review_of_Various_Maze

In [32]:
client_1.get_object(bucket_name="python-test-bucket-2", obj_name=objects[0], file_path=objects[0])

In [ ]:
client_1.create_object("python-test-bucket-2", "object-no-1", "daily_IBM.csv")

In [ ]:
client_1.get_object("python-test-bucket-2", "(The Practical Guides) Michael J. Donahoo, Gregory D. Speegle - SQL_ Practical Guide for Developers-Morgan Kaufmann (2005).pdf")

In [ ]:
client_1.delete_object("python-test-bucket-2", "(The Practical Guides) Michael J. Donahoo, Gregory D. Speegle - SQL_ Practical Guide for Developers-Morgan Kaufmann (2005).pdf")

1

In [ ]:
import looker_sdk
import os
import json

In [ ]:
!pip install "apache-airflow[celery]==2.9.3" --constraint "https://raw.githubusercontent.com/apache/airflow/constraints-2.9.3/constraints-3.8.txt"

^C


     ---------------------------------------- 0.0/43.8 kB ? eta -:--:--
     ----------------- -------------------- 20.5/43.8 kB 330.3 kB/s eta 0:00:01
     -------------------------- ----------- 30.7/43.8 kB 330.3 kB/s eta 0:00:01
     ----------------------------------- -- 41.0/43.8 kB 393.8 kB/s eta 0:00:01
     ----------------------------------- -- 41.0/43.8 kB 393.8 kB/s eta 0:00:01
     ----------------------------------- -- 41.0/43.8 kB 393.8 kB/s eta 0:00:01
     ----------------------------------- -- 41.0/43.8 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 43.8/43.8 kB 119.4 kB/s eta 0:00:00
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
     ---------------------------------------- 0.0/152.0 kB ? eta -:--:--
     ------- ------------------------------- 30.7/152.0 kB 1.3 MB/s eta 0:00:01
     ------------ ------------------------ 51.2/152.0 kB 871.5 kB/s eta 0:00:01
     ----------------- ------------------- 71.7/152.0 kB 787.7 kB/s eta

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'D:\\personal_project\\untitled\\.venv\\Lib\\site-packages\\airflow\\migrations\\versions\\0120_2_5_0_add_updated_at_to_dagrun_and_ti.py'
Check the permissions.



In [ ]:
!AIRFLOW_VERSION=2.0.1
!PYTHON_VERSION="$(python --version | cut -d " " -f 2 | cut -d "." -f 1-2)"
!CONSTRAINT_URL="https://raw.githubusercontent.com/apache/airflow/constraints-${AIRFLOW_VERSION}/constraints-${PYTHON_VERSION}.txt"
!pip install "apache-airflow[async,postgres,google]==${AIRFLOW_VERSION}" --constraint "${CONSTRAINT_URL}"

'AIRFLOW_VERSION' is not recognized as an internal or external command,
operable program or batch file.
'PYTHON_VERSION' is not recognized as an internal or external command,
operable program or batch file.


'CONSTRAINT_URL' is not recognized as an internal or external command,
operable program or batch file.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: '${CONSTRAINT_URL}'
